In [ ]:
!pip install lightgbm

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.metrics import roc_curve, make_scorer, roc_auc_score, f1_score,auc, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")
from scipy import stats

ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
train = pd.read_csv('./application_train.csv/application_train.csv')
train = train.loc[:100000,: ]
train.shape

In [ ]:
def group_and_plot(df, variable):
    grouped_by_target = df.groupby('TARGET')[variable].value_counts().reset_index(name='count')
    grouped_by_target.pivot("TARGET",variable ,"count").plot(kind='bar', figsize=(20,10))
    plt.xlabel("TARGET",size=20)
    plt.ylabel("number",size=20)
    plt.xticks(rotation = 0)
    plt.xticks(size=20)
    plt.title(variable + " / target" ,size=20)
    plt.yticks(size=20)
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
categorical = [  'FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5','FLAG_DOCUMENT_6',
                 'FLAG_DOCUMENT_7','FLAG_DOCUMENT_8','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10','FLAG_DOCUMENT_11',
                 'FLAG_DOCUMENT_12','FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16',
                 'FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21',
                 'REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION',
                 'LIVE_REGION_NOT_WORK_REGION','REG_CITY_NOT_LIVE_CITY',
                 'REG_CITY_NOT_WORK_CITY','LIVE_CITY_NOT_WORK_CITY','ORGANIZATION_TYPE','WEEKDAY_APPR_PROCESS_START',
                 'REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY','FLAG_MOBIL',
                 'FLAG_EMP_PHONE','FLAG_WORK_PHONE',
                 'FLAG_CONT_MOBILE','FLAG_PHONE','FLAG_EMAIL','OCCUPATION_TYPE','CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY','NAME_CONTRACT_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS','TARGET',
 'NAME_HOUSING_TYPE',
 'NAME_INCOME_TYPE','NAME_TYPE_SUITE','FONDKAPREMONT_MODE','HOUSETYPE_MODE','WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE'
              ]

numerical = ['SK_ID_CURR','CNT_CHILDREN','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY',
             'AMT_GOODS_PRICE','AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK',
             'AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR',
            'OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE',
            'OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE','DAYS_LAST_PHONE_CHANGE',
            'APARTMENTS_AVG',
            'BASEMENTAREA_AVG','YEARS_BEGINEXPLUATATION_AVG','YEARS_BUILD_AVG','COMMONAREA_AVG','ELEVATORS_AVG',
            'ENTRANCES_AVG','FLOORSMAX_AVG','FLOORSMIN_AVG','LANDAREA_AVG','LIVINGAPARTMENTS_AVG',
            'LIVINGAREA_AVG',
            'NONLIVINGAPARTMENTS_AVG','NONLIVINGAREA_AVG','APARTMENTS_MODE','BASEMENTAREA_MODE','YEARS_BEGINEXPLUATATION_MODE',
            'YEARS_BUILD_MODE','COMMONAREA_MODE','ELEVATORS_MODE',
            'ENTRANCES_MODE','FLOORSMAX_MODE','FLOORSMIN_MODE','LANDAREA_MODE','LIVINGAPARTMENTS_MODE',
            'LIVINGAREA_MODE',
            'NONLIVINGAPARTMENTS_MODE','NONLIVINGAREA_MODE','APARTMENTS_MEDI',
 'BASEMENTAREA_MEDI','YEARS_BEGINEXPLUATATION_MEDI','YEARS_BUILD_MEDI','COMMONAREA_MEDI',
 'ELEVATORS_MEDI','ENTRANCES_MEDI','FLOORSMAX_MEDI','FLOORSMIN_MEDI','LANDAREA_MEDI',
 'LIVINGAPARTMENTS_MEDI','LIVINGAREA_MEDI','NONLIVINGAPARTMENTS_MEDI','NONLIVINGAREA_MEDI','EXT_SOURCE_1','EXT_SOURCE_2',
 'EXT_SOURCE_3','DAYS_BIRTH','DAYS_EMPLOYED','CNT_FAM_MEMBERS',
 'DAYS_REGISTRATION','DAYS_ID_PUBLISH','OWN_CAR_AGE']

In [ ]:
def fillna_cat(col):
    train[col].value_counts().sort_values(ascending=False)[0]

In [ ]:
for col in categorical:
    train[col] = train[col].astype('category')

In [ ]:
for col in numerical:
    try:
        train[col] = train[col].astype(np.float64)
    except ValueError:
        print(col)
    train[col] = train[col].fillna(0)

In [ ]:
def check_categorical_nan():
    for col in categorical:
        if train[train[col].isna()].any().sum()>0:
            print("this has NAN:" + col)

In [ ]:
def check_numerical_nan():
    for col in numerical:
        if train[train[col].isna()].any().sum()>0:
            print("this numeric col has NAN:" + col)

In [ ]:
all_cols = numerical+categorical

In [ ]:
len(all_cols)

In [ ]:
train = train.loc[:, all_cols]

In [ ]:
most_popular = fillna_cat('OCCUPATION_TYPE')
train['OCCUPATION_TYPE'] = train['OCCUPATION_TYPE'].fillna(value= most_popular, method='ffill')

most_popular = fillna_cat('NAME_TYPE_SUITE')
train['NAME_TYPE_SUITE'] = train['NAME_TYPE_SUITE'].fillna(value= most_popular, method='ffill')

most_popular = fillna_cat('FONDKAPREMONT_MODE')
train['FONDKAPREMONT_MODE'] = train['FONDKAPREMONT_MODE'].fillna(value= most_popular, method='ffill')

most_popular = fillna_cat('HOUSETYPE_MODE')
train['HOUSETYPE_MODE'] = train['HOUSETYPE_MODE'].fillna(value= most_popular, method='ffill')

most_popular = fillna_cat('WALLSMATERIAL_MODE')
train['WALLSMATERIAL_MODE'] = train['WALLSMATERIAL_MODE'].fillna(value= most_popular, method='ffill')

most_popular = fillna_cat('EMERGENCYSTATE_MODE')
train['EMERGENCYSTATE_MODE'] = train['EMERGENCYSTATE_MODE'].fillna(value= most_popular, method='ffill')


In [ ]:
check_categorical_nan()

In [ ]:
check_numerical_nan()

In [ ]:
train.shape

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(train.corr())
plt.show()

In [ ]:
def remove_high_corr_var(df):
    corr_matrix = df.corr().abs()
    high_corr_var=np.where(corr_matrix>0.8)

    high_corr_var=[ (corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
    to_drop = list()
    for tupl in high_corr_var:
        to_drop.append(tupl[1])
    #to_drop.remove("TARGET")
    print(to_drop)
    df.drop(columns=to_drop, axis=1, inplace=True)

remove_high_corr_var(train)

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(train.corr())

In [ ]:
def get_numerical(df):
    numerical = list()
    for col in df.columns.tolist():
        if df[col].dtype == np.float64:
            numerical.append(col)
    return numerical
            
def get_categorical(df):
    categ = list()
    for col in df.columns.tolist():
        if df[col].dtype != np.float64:
            categ.append(col)
    return categ

In [ ]:
numerical = get_numerical(train)
categorical = get_categorical(train)

In [ ]:
numericals_to_draw_against = list(['CNT_CHILDREN','AMT_INCOME_TOTAL',
 'AMT_CREDIT','AMT_ANNUITY',
 'AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR',
 'OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE',
 'DAYS_LAST_PHONE_CHANGE','APARTMENTS_AVG',
 'BASEMENTAREA_AVG','YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG','COMMONAREA_AVG',
 'ENTRANCES_AVG','FLOORSMAX_AVG',
 'FLOORSMIN_AVG','LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG','NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG','EXT_SOURCE_1',
 'EXT_SOURCE_2','EXT_SOURCE_3',
 'DAYS_BIRTH','DAYS_EMPLOYED', 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE'])

In [ ]:
categoricals_to_draw_against = list(['REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION',
'LIVE_REGION_NOT_WORK_REGION','REG_CITY_NOT_LIVE_CITY',
'REG_CITY_NOT_WORK_CITY','LIVE_CITY_NOT_WORK_CITY','WEEKDAY_APPR_PROCESS_START',
'REGION_RATING_CLIENT',
'REGION_RATING_CLIENT_W_CITY','FLAG_MOBIL',
'FLAG_EMP_PHONE','FLAG_WORK_PHONE',
'FLAG_CONT_MOBILE','FLAG_PHONE','FLAG_EMAIL',
'CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY'])

In [ ]:
categoricals_to_draw_against_name_types = list(['NAME_CONTRACT_TYPE','OCCUPATION_TYPE','ORGANIZATION_TYPE',
'NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','TARGET','NAME_HOUSING_TYPE',
'NAME_INCOME_TYPE','NAME_TYPE_SUITE',
'FONDKAPREMONT_MODE','HOUSETYPE_MODE',
'WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE'])

In [ ]:
def grouper(train, var):
    
    qq = train.groupby('TARGET')[var].value_counts().sort_values(ascending=True).reset_index(name='count')
    rotate_x_labels=True
    if train[var].value_counts().nunique() < 4:
        rotate_x_labels=False
        
    #for visual details
    mainfontsize = 40
    bartextsize=30
    rotation_for_bar_text = 0
    format_for_bar_text = '.2f'
    figure_size_for_big_chart = (25,10)
    #decide how to draw many categories - select diff size, text size etc
    if len(qq) > 20:
        many_categories = True
        mainfontsize = 15
        bartextsize=10
        rotation_for_bar_text = 30
        format_for_bar_text = '.1f'
        figure_size_for_big_chart = (20,10)
        
    #compute percent rate 
    totals = list()
    for c in qq[var].values:
        totals.append(np.sum(qq[qq[var] == c]['count']))

    qq['totals'] = pd.Series(totals,index=qq.index)
    qq['percent_rate'] = (qq['count']*100) / qq['totals']
    
    fig, ax1 = plt.subplots(figsize=figure_size_for_big_chart)
    
    qq['TARGET'] = qq['TARGET'].apply(lambda status: rename_target(status))
    #df qq is sorted so bars are in descending order
    qq = qq.sort_values(by=['percent_rate'],ascending=False)
    barplot = sns.barplot(x=var,y='percent_rate',hue='TARGET',data=qq,
                palette='rainbow',saturation=0.5,
                hue_order=['YES', 'NO'],ax=ax1)
       
    for p in barplot.patches:
        barplot.annotate(format(p.get_height(), format_for_bar_text), 
                               (p.get_x() + p.get_width() / 2., 
                                p.get_height()), color = 'black', fontsize=bartextsize,
                                 rotation=rotation_for_bar_text,
                                 ha = 'center', va = 'center', 
                                 xytext = (0, 10), 
                                 textcoords = 'offset points')
    
    plt.xlabel(s=var,size=40)
    plt.ylabel(s= '%',size=40)
    
    if var == 'ORGANIZATION_TYPE':
        plt.xticks(rotation=90,size=mainfontsize)    
        
    elif rotate_x_labels:
        plt.xticks(rotation=45,size=mainfontsize)    
    else:
        plt.xticks(size=mainfontsize)
       
    plt.yscale('log')
    plt.yticks(size=20)
    plt.title(var + " vs " + " % count",size=40)
    plt.legend(loc='center', prop={'size':30})
    plt.show()
    print()
    print()
    print()
    return qq


w = grouper(train, 'NAME_INCOME_TYPE')
w = grouper(train, 'ORGANIZATION_TYPE')

In [ ]:
for col in categoricals_to_draw_against:
    grouper(train, col)

In [ ]:
for col in categoricals_to_draw_against_name_types:
    try:
        grouper(train, col)
    except ValueError:
        pass

In [ ]:
def rename_target(status):
    if status == 1:
        return 'YES'
    else:
        return 'NO'

In [ ]:
def plot_N_kde_plots(df, feature_list):
    cmap = sns.palplot(sns.color_palette("Paired"))
    # plot N features
    plt.figure(figsize=(20,10))
    for feature in feature_list:                
        plt.yscale('log')
        p1=sns.kdeplot(df[feature], shade=True, color=cmap)
    plt.show()

# Using PyOD to outlier detection

# draw pairplots

In [ ]:
g = sns.pairplot(train.iloc[:,:10], diag_kind="kde", markers="+",
                  plot_kws=dict(s=50, edgecolor="b", linewidth=1),
                  diag_kws=dict(shade=True))

# draw box plots

In [ ]:
def plot_box_plot(x_boxen, y_boxen, df):
    plt.figure(figsize=(20,10))
    ax = sns.boxplot(x=x_boxen, y=y_boxen, hue="TARGET",
                      data=df, palette="Set3")
    plt.yticks(fontsize=20)
    plt.xticks(fontsize=10)
    plt.xlabel(fontsize=10, s= col)
    
    plt.title(x_boxen + ' vs ' + y_boxen,size=20)
    plt.show()

In [ ]:
for categ in categoricals_to_draw_against:
    for numer in numericals_to_draw_against:
        plot_box_plot(categ, numer, train)

In [ ]:
for categ in categoricals_to_draw_against_name_types:
    for numer in numericals_to_draw_against:
        plot_box_plot(categ, numer, train)

# drawing distplots

In [ ]:
def plot_distplots(col,df, scaling=False):    
    plt.figure(figsize=(20,10))    
    sns.distplot(df[col],color='r')    
    plt.yticks(fontsize=20)
    plt.xlabel(fontsize=10, s= col)
    if scaling:
        plt.yscale('log')
    plt.xticks(fontsize=10)
    plt.title(col,size=20)
    plt.show()

In [ ]:
for col in numericals_to_draw_against:
    plot_distplots(col, train)

# drawing all boxen plots

In [ ]:
def boxen_plot_chart(x_boxen, y_boxen,df, scaling=False, name_type = False):    
    plt.figure(figsize=(20,10))    
    ax = sns.boxenplot(x=x_boxen, y=y_boxen, data=df)
    if scaling:
        plt.yscale('log')
    plt.xticks(fontsize=20)
    plt.title(x_boxen + " vs " + y_boxen,size=20)
    plt.show()

In [ ]:
for categ in categoricals_to_draw_against:
    for numer in numericals_to_draw_against:
        boxen_plot_chart(categ, numer, train)

In [ ]:
for categ in categoricals_to_draw_against_name_types:
    for numer in numericals_to_draw_against:
        boxen_plot_chart(categ, numer, train, name_type=True)

# drawing all stripplots

In [ ]:
def strip_plot_chart(x_boxen, y_boxen,df, scaling=False, name_type=False, show_hue = False):    
    plt.figure(figsize=(20,10))
    plt.yticks(fontsize=20)
    if scaling:
        plt.yscale('log')
    if show_hue:
        ax = sns.stripplot(x=x_boxen, y=y_boxen, data=df,hue='TARGET', jitter=0.05)
    else:
        ax = sns.stripplot(x=x_boxen, y=y_boxen, data=df, jitter=0.05)
    plt.xticks(rotation=60,fontsize=20)
    plt.title(x_boxen + " vs " + y_boxen,size=20)
    plt.show()

In [ ]:
for categ in categoricals_to_draw_against:
    for numer in numericals_to_draw_against:
        strip_plot_chart(categ, numer, train)

In [ ]:
for categ in categoricals_to_draw_against_name_types:
    for numer in numericals_to_draw_against:
        strip_plot_chart(categ, numer, train, name_type=True)

In [ ]:
for categ in categoricals_to_draw_against_name_types:
    for numer in numericals_to_draw_against:
        strip_plot_chart(categ, numer, train, name_type=True,show_hue=True)

In [ ]:
plt.figure(figsize=(20,10))
sns.set(style="ticks", color_codes=True)
g = sns.FacetGrid(train, col="TARGET", row="NAME_EDUCATION_TYPE",size=10)
g = g.map(plt.hist, 'AMT_ANNUITY',  color = 'r')
#plt.ylim(0,4000)
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
sns.set(style="ticks", color_codes=True)
g = sns.FacetGrid(train, col="TARGET", row="OCCUPATION_TYPE")
g = g.map(plt.hist, "AMT_CREDIT",color = 'r')
plt.ylim(0,4000)
plt.show()

# group and plot function plots()

In [ ]:
for col in categoricals_to_draw_against:
    group_and_plot(train, col)

In [ ]:
for col in categoricals_to_draw_against_name_types:
    try:
        group_and_plot(train, col)
    except ValueError:
        pass

# drawing JOINt grid

In [ ]:
def joint_dist_scatter(x_col, y_col, df):
#     plt.figure(figsize=(20,10))    
#     plt.yticks(fontsize=20)
#     plt.xticks(fontsize=10)
#     plt.xlabel(fontsize=10, s= col)
#     plt.title(x_col + ' vs ' + y_col,size=20)
    
    g = sns.JointGrid(x=x_col, y=y_col, data=df,size=6)
    g = g.plot_joint(plt.scatter, color=".5", edgecolor="black", marker='*')    
    g = g.plot_marginals(sns.distplot, kde=False, color="blue")
    g = g.plot_marginals(sns.kdeplot, color="green")    
    plt.show()

In [ ]:
len(numerical)

In [ ]:
for numer1 in numericals_to_draw_against:
    for numer2 in numericals_to_draw_against:
        if numer1 != numer2:
            joint_dist_scatter(numer1, numer2, train)

In [ ]:
for col in categoricals_to_draw_against:
    plt.figure(figsize=(20,10))
    sns.countplot(train[col])

In [ ]:
for col in categoricals_to_draw_against_name_types:
    plt.figure(figsize=(20,10))
    sns.countplot(train[col])
    plt.yticks(fontsize=20)
    plt.xlabel(fontsize=20, s= col)
    plt.ylabel(fontsize=20, s = 'count')
    plt.xticks(fontsize=20)
    
    plt.title(col,size=20)
    plt.show()


# scatter plots

In [ ]:
def plot_scatter(x_col, y_col, df, scale = False):
    plt.figure(figsize=(15,10))
    sns.set_palette("Paired")
    if scale:
        plt.yscale('log')
    sns.scatterplot(x=x_col, y=y_col,marker='X', hue='TARGET',linewidth=0,data=train)    
    plt.show()

In [ ]:
for numer1 in numericals_to_draw_against:
    for numer2 in numericals_to_draw_against:
        if numer1 != numer2:            
            plot_scatter(numer1, numer2, train, scale=True)

In [ ]:
plt.figure(figsize=(20,10))
plt.xticks(size=20)
sns.distplot(train['AMT_CREDIT'],bins=10)

In [ ]:
plt.figure(figsize=(20,10))
plt.xticks(size=20)
plt.xlim(-0.5,5)
plt.ylim(0,4)
sns.distplot(train['CNT_CHILDREN'],bins=3)

In [ ]:
for col in train.columns.tolist():
    fig = plt.figure(figsize=(20,10))
    train['CODE_GENDER'].value_counts().plot(kind='bar')
    plt.xticks(size=20)
    plt.yticks(size=20)
    plt.show()

In [ ]:
plt.figure(figsize=(20,10))
g = sns.JointGrid(x="AMT_INCOME_TOTAL", y="TARGET", data=train,
                   height=5, ratio=2)
g = g.plot_joint(sns.kdeplot, cmap="Reds_d")
g = g.plot_marginals(sns.kdeplot, color="r", shade=True)
plt.show()

In [ ]:
g = sns.JointGrid(x="AMT_INCOME_TOTAL", y="TARGET", data=train,
                   height=5, ratio=2)
g = g.plot_joint(sns.kdeplot, cmap="Reds_d")
g = g.plot_marginals(sns.kdeplot, color="r", shade=True)

In [ ]:
fig,ax = plt.subplots(1,2, figsize=(20,10))
ax1 = train['NAME_CONTRACT_TYPE'].value_counts().plot(kind='bar',ax= ax[0])
ax1.set_x

train['NAME_EDUCATION_TYPE'].value_counts().plot(kind='bar',ax= ax[1])

plt.show()